In [ ]:
import time
import datetime
start = time.time()

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import bbknn
from pandas.core.index import RangeIndex
from matplotlib import rcParams
from matplotlib.pyplot import rc_context

In [ ]:
sc.settings.verbosity = 3  
sc.logging.print_header() 
sc.settings.set_figure_params(dpi=80)

In [ ]:
V2IA = sc.read_loom('./V2h_Amp.loom')
V2IU = sc.read_loom('./V2h_Isth.loom')
P2IA = sc.read_loom('./P2h_Amp.loom')
P2IU = sc.read_loom('./P2h_Isth.loom')
V24IA = sc.read_loom('./V24Am.loom')
V24IU = sc.read_loom('./V24Is.loom')
P24IA = sc.read_loom('./P24h_Amp.loom')
P24IU = sc.read_loom('./P24h_Isth.loom')

In [ ]:
V2IA.var_names_make_unique()
V2IU.var_names_make_unique()
P2IA.var_names_make_unique()
P2IU.var_names_make_unique()
V24IA.var_names_make_unique()
V24IU.var_names_make_unique()
P24IA.var_names_make_unique()
P24IU.var_names_make_unique()


In [ ]:
V2IA.write('./V2hInfAmp.h5ad')

In [ ]:
V2IU.write('./V2hIsthUTJ.h5ad')

In [ ]:
P2IA.write('./P2hInfAmp.h5ad')

In [ ]:
P2IU.write('./P2hIsthUTJ.h5ad')

In [ ]:
V24IA.write('./V24hInfAmp.h5ad')

In [ ]:
V24IU.write('./V24hIsthUTJ.h5ad')

In [ ]:
P24IA.write('./P24hInfAmp.h5ad')

In [ ]:
P24IU.write('./P24hIsthUTJ.h5ad')

In [ ]:
V2IAIU = V2IA.concatenate(V2IU, batch_key='Region', batch_categories=['InfAmp','IsthUTJ'])
P2IAIU = P2IA.concatenate(P2IU, batch_key='Region', batch_categories=['InfAmp','IsthUTJ'])
V24IAIU = V24IA.concatenate(V24IU, batch_key='Region', batch_categories=['InfAmp','IsthUTJ'])
P24IAIU = P24IA.concatenate(P24IU, batch_key='Region', batch_categories=['InfAmp','IsthUTJ'])

In [ ]:
VP2hIAIU = V2IAIU.concatenate(P2IAIU, batch_key='Treatment', batch_categories=['V','P'])
VP24hIAIU = V24IAIU.concatenate(P24IAIU, batch_key='Treatment', batch_categories=['V','P'])

In [ ]:
VP2h24hIAIU = VP2hIAIU.concatenate(VP24hIAIU, batch_key='Time', batch_categories=['2h','24h'])

In [ ]:
VP2h24hIAIU

In [ ]:
VP2h24hIAIU.write('./VP2h24hIAIU.h5ad')

In [ ]:

sc.settings.verbosity = 
sc.logging.print_header() 
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
!pwd

In [ ]:
VP2h24hIAIU = sc.read( './VP2h24hIAIU.h5ad') 

In [ ]:
VP2h24hIAIU.var_names_make_unique()

In [ ]:
VP2h24hIAIU

In [ ]:

sc.pl.highest_expr_genes(VP2h24hIAIU, n_top=20)

In [ ]:
sc.pp.filter_cells(VP2h24hIAIU, min_genes=200) 
sc.pp.filter_genes(VP2h24hIAIU, min_cells=3)

In [ ]:

mito_genes = VP2h24hIAIU.var_names.str.startswith('mt') 
VP2h24hIAIU.obs['pct_counts_mt'] = np.sum(
    VP2h24hIAIU[:, mito_genes].X, axis=1).A1 / np.sum(VP2h24hIAIU.X, axis=1).A1 
VP2h24hIAIU.obs['total_counts'] = VP2h24hIAIU.X.sum(axis=1).A1 

In [ ]:
VP2h24hIAIU

In [ ]:

sc.pl.violin(VP2h24hIAIU, ['n_genes'], jitter=0.4)
sc.pl.violin(VP2h24hIAIU, ['total_counts'], jitter=0.4)
sc.pl.violin(VP2h24hIAIU, ['pct_counts_mt'], jitter=0.4)

In [ ]:
VP2h24hIAIU

In [ ]:
sc.pl.scatter(VP2h24hIAIU, x='total_counts', y='pct_counts_mt')
# sc.pl.scatter(VP2h24hIAIU, x='total_counts', y='n_genes_by_counts')
sc.pl.scatter(VP2h24hIAIU, x='total_counts', y='n_genes')

In [ ]:
VP2h24hIAIU

In [ ]:
VP2h24hIAIU = VP2h24hIAIU[VP2h24hIAIU.obs.n_genes < 4000, :]
VP2h24hIAIU = VP2h24hIAIU[VP2h24hIAIU.obs.pct_counts_mt < 0.1, :]

In [ ]:
sc.pp.normalize_total(VP2h24hIAIU, target_sum=None)

In [ ]:
sc.pp.log1p(VP2h24hIAIU)

In [ ]:
sc.pp.regress_out(VP2h24hIAIU, ['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pp.highly_variable_genes(VP2h24hIAIU, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(VP2h24hIAIU)

In [ ]:
VP2h24hIAIU.raw = VP2h24hIAIU

In [ ]:
VP2h24hIAIU = VP2h24hIAIU[:, VP2h24hIAIU.var.highly_variable]

In [ ]:
VP2h24hIAIU.write('./CheckPoint.h5ad')
CheckPoint = sc.read('./CheckPoint.h5ad')

In [ ]:
sc.pp.scale(VP2h24hIAIU, max_value=10) 

In [ ]:

sc.tl.pca(VP2h24hIAIU, svd_solver='arpack')

In [ ]:
sc.pl.pca(VP2h24hIAIU, color='Treatment')

In [ ]:

sc.pl.pca(VP2h24hIAIU, color='Time')

In [ ]:
sc.pl.pca_variance_ratio(VP2h24hIAIU, log=True)

In [ ]:
sc.pl.pca(VP2h24hIAIU, wspace=0.35,
          color=['Treatment', 'Time', 'Region', 'Epcam', 'Ovgp1', 'Foxj1', 'Myh11', 'Cd52', 'Pecam1', 
                  
                  'Pdgfra', 'Twist2', 'Igf1', 'Dcn', 'Serpina1e', 'Myb', 'Pax8', 'Pax2', 'Cxcl17', 'Id2', 'Pdxk', 'Ccdc153'])

In [ ]:
sc.pl.pca_variance_ratio(VP2h24hIAIU, n_pcs=40, log=True)

In [ ]:
VP2h24hIAIU

In [ ]:

sc.pp.neighbors(VP2h24hIAIU, n_neighbors=10, n_pcs=40)

In [ ]:

sc.tl.leiden(VP2h24hIAIU)
sc.tl.paga(VP2h24hIAIU)
sc.pl.paga(VP2h24hIAIU, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(VP2h24hIAIU, init_pos='paga')

In [ ]:
sc.tl.umap(VP2h24hIAIU)

In [ ]:

sc.tl.leiden(VP2h24hIAIU, resolution=0.2) 

In [ ]:
sc.pl.umap(VP2h24hIAIU,color=['leiden','Treatment', 'Time', 'Region'],legend_loc='on data')

In [ ]:
sc.pl.umap(VP2h24hIAIU,
           color=['leiden', 'Treatment', 'Time', 'Region', 'Epcam', 'Ovgp1', 'Pax8', 'Pax2', 'Foxj1', 'Ccdc153', 
                  'Myh11', 'Cd52', 'Ptprc', 'Pecam1', 
                  
                  'Pdgfra', 'Twist2', 'Igf1', 'Dcn', 'Serpina1e', 'Myb', 'Cxcl17', 'Id2', 'Pdxk'])

In [ ]:
VP2h24hIAIU.obs.groupby(["leiden","Treatment","Time"]).apply(len)

In [ ]:
VP2h24hIAIU.obs.groupby(["leiden","Treatment"]).apply(len)

In [ ]:
VP2h24hIAIU.obs.groupby(["leiden","Time"]).apply(len)

In [ ]:
VP2h24hIAIU.obs.groupby(["leiden","Region"]).apply(len)

In [ ]:
   
VP2h24hIAIU_batchcorrected = bbknn.bbknn(VP2h24hIAIU, batch_key='Time', copy=True)

In [ ]:

sc.pl.pca(VP2h24hIAIU_batchcorrected, wspace=0.35,
          color=['leiden', 'Treatment', 'Time', 'Region', 'Epcam', 'Ovgp1', 'Foxj1', 'Myh11', 'Cd52', 'Pecam1', 
                 
                  'Pdgfra', 'Twist2', 'Igf1', 'Dcn', 'Serpina1e', 'Myb', 'Pax8', 'Pax2', 'Cxcl17', 'Id2', 'Pdxk', 'Ccdc153'])

In [ ]:
sc.tl.umap(VP2h24hIAIU_batchcorrected)

In [ ]:
sc.pl.umap(VP2h24hIAIU_batchcorrected, wspace=0.35, 
           color=['leiden', 'Region', 'Time', 'Treatment', 'Epcam', 'Ovgp1', 'Foxj1', 'Myh11', 'Cd52', 'Pecam1', 
                  
                  'Pdgfra', 'Twist2', 'Igf1', 'Dcn', 'Serpina1e', 'Myb', 'Pax8', 'Pax2', 'Cxcl17', 'Id2', 'Pdxk', 
                  'Ccdc153', 'Ptprc'])

In [ ]:
marker_genes = ['Epcam', 'Krt8','Krt19','Vim','Pdgfra','Twist2', 'Sprr2f',
                'Acta2', 'Myh11','Foxj1', 'Ccdc153', 'Ovgp1','Wt1','Serpina1e',
                'S100g','Pecam1', 'Cd52', 'Pgr', 'Esr1', 'Crabp2', 'Cd44', 'Pdxk'
                 ]

In [ ]:
sc.pl.umap(VP2h24hIAIU_batchcorrected, color=marker_genes)

In [ ]:
VP2h24hIAIU_batchcorrected.obs.groupby(["leiden","Treatment","Time","Region"]).apply(len)

In [ ]:
VP2h24hIAIU_batchcorrected.obs.groupby(["leiden","Treatment"]).apply(len)

In [ ]:
new_cluster_names = [
     '0 Secretory_InfAmp',
     '1 Secretory_IsthUTJ',
     '2 Ciliated_InfAmp',
     '3 Fibroblast_Pdgfra+',
     '4 Fibroblast_Pdgfra-',
     '5 Secretory+Ciliated_Veh',
     '6 Secretory_IsthUTJ_subset1',
     '7 Immune1',
     '8 Cliated_IsthUTJ1',
     '9 Cliated_IsthUTJ2',
     '10 Endothelial1',
     '11 Secretory_IsthUTJ_subset2',
     '12 Muscle',
     '13 Immune2',
    '14 Endothelial2'
]
VP2h24hIAIU_batchcorrected.rename_categories('leiden', new_cluster_names)

In [ ]:
plt.figure(figsize=(8, 2))
for i in range(28):
    plt.scatter(i, 1, c=sc.pl.palettes.zeileis_28[i], s=200)
plt.show()

In [ ]:
zeileis_colors = np.array(sc.pl.palettes.zeileis_28)
new_colors = np.array(VP2h24hIAIU_batchcorrected.uns['leiden_colors'])

In [ ]:
new_colors[[0]] = zeileis_colors[[0]]  
new_colors[[1]] = zeileis_colors[[7]]  
new_colors[[2]] = zeileis_colors[[5]]
new_colors[[3]] = zeileis_colors[[16]]  
new_colors[[4]] = zeileis_colors[[17]] 
new_colors[[5]] = zeileis_colors[[6]] 
new_colors[[6]] = zeileis_colors[[18]] 
new_colors[[7]] = zeileis_colors[[12]] 
new_colors[[8]] = zeileis_colors[[4]] 
new_colors[[9]] = zeileis_colors[[3]] 
new_colors[[10]] = zeileis_colors[[26]] 
new_colors[[11]] = zeileis_colors[[19]] 
new_colors[[12]] = zeileis_colors[[24]] 
new_colors[[13]] = zeileis_colors[[13]] 
new_colors[[14]] = zeileis_colors[[27]] 

In [ ]:
VP2h24hIAIU_batchcorrected.uns['leiden_colors'] = new_colors

In [ ]:
sc.pl.umap(VP2h24hIAIU_batchcorrected, color='leiden', legend_loc='right margin', legend_fontsize='8.0',
           frameon=False, save='VP2h24hIAIU_AllPops_Leiden_060724.pdf')

In [ ]:
sc.pl.umap(VP2h24hIAIU_batchcorrected, color='Time', legend_loc='right margin', legend_fontsize='8.0',
           frameon=False, use_raw=False, save='VP2h24hIAIU_AllPops_Time_060724.pdf')

In [ ]:
new_cluster_names = [
    '2h',
    '24h'
]
VP2h24hIAIU_batchcorrected.rename_categories('Time', new_cluster_names)

In [ ]:
plt.figure(figsize=(8, 2))
for i in range(28):
    plt.scatter(i, 1, c=sc.pl.palettes.zeileis_28[i], s=200)
plt.show()

In [ ]:
zeileis_colors = np.array(sc.pl.palettes.zeileis_28)
new_colors = np.array(VP2h24hIAIU_batchcorrected.uns['Time_colors'])

In [ ]:
new_colors[[0]] = zeileis_colors[[6]]  #0 2h
new_colors[[1]] = zeileis_colors[[17]]  #1 24h

In [ ]:
VP2h24hIAIU_batchcorrected.uns['Time_colors'] = new_colors

In [ ]:
sc.pl.umap(VP2h24hIAIU_batchcorrected, color='Time', legend_loc='right margin', legend_fontsize='8.0',
           frameon=False, save='VP2h24hIAIU_AllPops_Time_060724.pdf')

In [ ]:
new_cluster_names = [
    'InfAmp',
    'IsthUTJ'
]
VP2h24hIAIU_batchcorrected.rename_categories('Region', new_cluster_names)

In [ ]:
plt.figure(figsize=(8, 2))
for i in range(28):
    plt.scatter(i, 1, c=sc.pl.palettes.zeileis_28[i], s=200)
plt.show()

In [ ]:
zeileis_colors = np.array(sc.pl.palettes.zeileis_28)
new_colors = np.array(VP2h24hIAIU_batchcorrected.uns['Region_colors'])

In [ ]:
new_colors[[0]] = zeileis_colors[[9]]  #0 InfAmp
new_colors[[1]] = zeileis_colors[1]  #1 IsthUTJ

In [ ]:
VP2h24hIAIU_batchcorrected.uns['Region_colors'] = new_colors

In [ ]:
sc.pl.umap(VP2h24hIAIU_batchcorrected, color='Region', legend_loc='right margin', legend_fontsize='8.0',
           frameon=False, save='VP2h24hIAIU_AllPops_Region_060724.pdf')

In [ ]:
new_cluster_names = [
    'Veh',
    'P4'
]
VP2h24hIAIU_batchcorrected.rename_categories('Treatment', new_cluster_names)

In [ ]:
plt.figure(figsize=(8, 2))
for i in range(28):
    plt.scatter(i, 1, c=sc.pl.palettes.zeileis_28[i], s=200)
plt.show()

In [ ]:
zeileis_colors = np.array(sc.pl.palettes.zeileis_28)
new_colors = np.array(VP2h24hIAIU_batchcorrected.uns['Treatment_colors'])

In [ ]:
new_colors[[0]] = zeileis_colors[[2]]  #0 Veh
new_colors[[1]] = zeileis_colors[5]  #1 P4

In [ ]:
VP2h24hIAIU_batchcorrected.uns['Treatment_colors'] = new_colors

In [ ]:
sc.pl.umap(VP2h24hIAIU_batchcorrected, color='Treatment', legend_loc='right margin', legend_fontsize='8.0',
           frameon=False, save='VP2h24hIAIU_AllPops_Treatment_060724.pdf')

In [ ]:
ax = sc.pl.dotplot(VP2h24hIAIU_batchcorrected, 
                   marker_genes, groupby='leiden', dendrogram=True, color_map='Blues',
                  save='VP2h24hIAIU_AllPops_MarkerDotPlot_060724')

In [ ]:
ax = sc.pl.dotplot(VP2h24hIAIU_batchcorrected, 
                   marker_genes, groupby='Region', dendrogram=True, color_map='Blues',
                  save='VP2h24hIAIU_AllPops_MarkerDotPlot_Region_060724')

In [ ]:
ax = sc.pl.dotplot(VP2h24hIAIU_batchcorrected, 
                   marker_genes, groupby='Treatment', dendrogram=True, color_map='Blues',
                  save='VP2h24hIAIU_AllPops_MarkerDotPlot_Treatment_060724')

In [ ]:
sc.tl.rank_genes_groups(VP2h24hIAIU_batchcorrected, 'Treatment', method='wilcoxon',n_genes=1000)
sc.pl.rank_genes_groups(VP2h24hIAIU_batchcorrected, n_genes=1000, sharey=False)
df = pd.DataFrame(VP2h24hIAIU_batchcorrected.uns['rank_genes_groups']['names']).head(1000)
df.to_csv('./VP2h24hIAIU_AllPops_Treatment_top1000.csv')

In [ ]:

VP2h24hIAIU_batchcorrected_Epithelial = VP2h24hIAIU_batchcorrected[VP2h24hIAIU_batchcorrected.obs['leiden'].isin(['0 Secretory_InfAmp',
                                                                                                                  '1 Secretory_IsthUTJ',
                                                                                                                  '2 Ciliated_InfAmp',
                                                                                                                  '5 Secretory+Ciliated_Veh',
                                                                                                                  '6 Secretory_IsthUTJ_Subset1',
                                                                                                                  '8 Ciliated_IsthUTJ1'
                                                                                                                 '9 Ciliated_IsthUTJ2',
                                                                                                                 '11 Secretory_IsthUTJ_subset2'])]

VP2h24hIAIU_batchcorrected_Secretory = VP2h24hIAIU_batchcorrected[VP2h24hIAIU_batchcorrected.obs['leiden'].isin(['0 Secretory_InfAmp',
                                                                                                                  '1 Secretory_IsthUTJ',
                                                                                                                  '5 Secretory+Ciliated_Veh',
                                                                                                                  '6 Secretory_IsthUTJ_Subset1',
                                                                                                                 '11 Secretory_IsthUTJ_subset2'])]

VP2h24hIAIU_batchcorrected_Ciliated = VP2h24hIAIU_batchcorrected[VP2h24hIAIU_batchcorrected.obs['leiden'].isin(['2 Ciliated_InfAmp',
                                                                                                                  '8 Ciliated_IsthUTJ1'
                                                                                                                 '9 Ciliated_IsthUTJ2'])]

VP2h24hIAIU_batchcorrected_Fibroblast = VP2h24hIAIU_batchcorrected[VP2h24hIAIU_batchcorrected.obs['leiden'].isin(['3 Fibroblast_Pdgfra+',
                                                                                                                  '4 Fibroblast_Pdgfra-'])]

VP2h24hIAIU_batchcorrected_Muscle = VP2h24hIAIU_batchcorrected[VP2h24hIAIU_batchcorrected.obs['leiden'].isin(['12 Muscle'])]

VP2h24hIAIU_batchcorrected_Immune = VP2h24hIAIU_batchcorrected[VP2h24hIAIU_batchcorrected.obs['leiden'].isin(['7 Immune1','13 Immune2'])]


In [ ]:
VP2h24hIAIU_batchcorrected_Epithelial

In [ ]:
VP2h24hIAIU_batchcorrected_Secretory

In [ ]:
VP2h24hIAIU_batchcorrected_Ciliated

In [ ]:
VP2h24hIAIU_batchcorrected_Fibroblast

In [ ]:
VP2h24hIAIU_batchcorrected_Muscle

In [ ]:
VP2h24hIAIU_batchcorrected_Immune

In [ ]:
sc.tl.pca(VP2h24hIAIU_batchcorrected_Epithelial, svd_solver='arpack')
sc.tl.pca(VP2h24hIAIU_batchcorrected_Secretory, svd_solver='arpack')
sc.tl.pca(VP2h24hIAIU_batchcorrected_Ciliated, svd_solver='arpack')
sc.tl.pca(VP2h24hIAIU_batchcorrected_Immune, svd_solver='arpack')
sc.tl.pca(VP2h24hIAIU_batchcorrected_Fibroblast, svd_solver='arpack')
sc.tl.pca(VP2h24hIAIU_batchcorrected_Muscle, svd_solver='arpack')

In [ ]:
sc.pl.pca_variance_ratio(VP2h24hIAIU_batchcorrected_Epithelial, n_pcs=40, log=True)
sc.pl.pca_variance_ratio(VP2h24hIAIU_batchcorrected_Secretory, n_pcs=40, log=True)
sc.pl.pca_variance_ratio(VP2h24hIAIU_batchcorrected_Ciliated, n_pcs=40, log=True)
sc.pl.pca_variance_ratio(VP2h24hIAIU_batchcorrected_Immune, n_pcs=40, log=True)
sc.pl.pca_variance_ratio(VP2h24hIAIU_batchcorrected_Fibroblast, n_pcs=40, log=True)
sc.pl.pca_variance_ratio(VP2h24hIAIU_batchcorrected_Muscle, n_pcs=40, log=True)

In [ ]:
sc.pp.neighbors(VP2h24hIAIU_batchcorrected_Epithelial, n_neighbors=10, n_pcs=40)
sc.pp.neighbors(VP2h24hIAIU_batchcorrected_Secretory, n_neighbors=10, n_pcs=40)
sc.pp.neighbors(VP2h24hIAIU_batchcorrected_Ciliated, n_neighbors=10, n_pcs=40)
sc.pp.neighbors(VP2h24hIAIU_batchcorrected_Immune, n_neighbors=10, n_pcs=40)
sc.pp.neighbors(VP2h24hIAIU_batchcorrected_Muscle, n_neighbors=10, n_pcs=40)
sc.pp.neighbors(VP2h24hIAIU_batchcorrected_Fibroblast, n_neighbors=10, n_pcs=40)

In [ ]:
sc.tl.umap(VP2h24hIAIU_batchcorrected_Epithelial)
sc.tl.umap(VP2h24hIAIU_batchcorrected_Secretory)
sc.tl.umap(VP2h24hIAIU_batchcorrected_Ciliated)
sc.tl.umap(VP2h24hIAIU_batchcorrected_Immune)
sc.tl.umap(VP2h24hIAIU_batchcorrected_Muscle)
sc.tl.umap(VP2h24hIAIU_batchcorrected_Fibroblast)

In [ ]:
gene_dict = ['Epcam', 'Ovgp1', 'Foxj1', 'Myh11', 'Cd52', 'Ptprc', 'Pecam1', 
                  
                  'Pdgfra', 'Twist2', 'Igf1', 'Dcn', 'Serpina1e', 'Myb', 'Pax8', 'Pax2', 'Cxcl17', 'Id2', 'Pdxk',
             'Ccdc153', 'Il22ra1', 'Il22ra2', 'Stat3']

In [ ]:
sc.pl.umap(VP2h24hIAIU_batchcorrected_Epithelial, color=gene_dict)
sc.pl.umap(VP2h24hIAIU_batchcorrected_Secretory, color=gene_dict)
sc.pl.umap(VP2h24hIAIU_batchcorrected_Ciliated, color=gene_dict)
sc.pl.umap(VP2h24hIAIU_batchcorrected_Fibroblast, color=gene_dict)
sc.pl.umap(VP2h24hIAIU_batchcorrected_Muscle, color=gene_dict)
sc.pl.umap(VP2h24hIAIU_batchcorrected_Immune, color=gene_dict)

In [ ]:
sc.pl.dotplot(VP2h24hIAIU_batchcorrected_Immune, ['Il1b', 'Il10', 'Il15', 'Il18', 'Ifng', 'Tlr2', 'Tlr4'],
groupby='Treatment', color_map='Blues', 
             save='_VP2h24hIAIU_batchcorrected_ImmuneCells_immune genes.pdf')

In [ ]:
sc.pl.dotplot(VP2h24hIAIU_batchcorrected, ['Il1b', 'Il10', 'Il15', 'Il18', 'Ifng', 'Tlr2', 'Tlr4'],
groupby='Treatment', color_map='Blues', 
             save='_VP2h24hIAIU_AllPops_immune genes.pdf')

In [ ]:
sc.pl.dotplot(VP2h24hIAIU_batchcorrected_Epithelial, ['Il1b', 'Il10', 'Il15', 'Il18', 'Ifng', 'Tlr2', 'Tlr4'],
groupby='Treatment', color_map='Blues', 
             save='_VP2h24hIAIU_EpithelialOnly_immune genes.pdf')

In [ ]:
sc.pl.dotplot(VP2h24hIAIU_batchcorrected_Secretory, ['Il1b', 'Il10', 'Il15', 'Il18', 'Ifng', 'Tlr2', 'Tlr4'],
groupby='Treatment', color_map='Blues', 
             save='_VP2h24hIAIU_SecretoryOnly_immune genes.pdf')

In [ ]:
sc.pl.dotplot(VP2h24hIAIU_batchcorrected_Ciliated, ['Il1b', 'Il10', 'Il15', 'Il18', 'Ifng', 'Tlr2', 'Tlr4'],
groupby='Treatment', color_map='Blues', 
             save='_VP2h24hIAIU_CiliatedOnly_immune genes.pdf')

In [ ]:
sc.pl.dotplot(VP2h24hIAIU_batchcorrected_Fibroblast, ['Il1b', 'Il10', 'Il15', 'Il18', 'Ifng', 'Tlr2', 'Tlr4'],
groupby='Treatment', color_map='Blues', 
             save='_VP2h24hIAIU_FibroblastOnly_immune genes.pdf')

In [ ]:
sc.pl.dotplot(VP2h24hIAIU_batchcorrected, ['Il10ra', 'Il10rb', 'Stat1', 'Stat3', 'Map2k1', 'Mapk8', 'Mapk14', 'Jak1', 'Tyk2'],
groupby='Treatment', color_map='Blues', 
             save='_VP2h24hIAIU_AllPops_IL22SignalingPathway.pdf')

In [ ]:
sc.pl.dotplot(VP2h24hIAIU_batchcorrected_Immune, ['Il10ra', 'Il10rb', 'Stat1', 'Stat3', 'Map2k1', 'Mapk8', 'Mapk14', 'Jak1', 'Tyk2'],
groupby='Treatment', color_map='Blues', 
             save='_VP2h24hIAIU_batchcorrected_ImmuneCells_IL22SignalingPathway.pdf')